# **Libraries  & Data set**

In [21]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report


In [ ]:
df = pd.read_csv("Alphabets_data.csv")

In [ ]:
df

,letter,xbox,ybox,width,height,onpix,xbar,ybar,x2bar,y2bar,xybar,x2ybar,xy2bar,xedge,xedgey,yedge,yedgex
0,T,2,8,3,5,1,8,13,0,6,6,10,8,0,8,0,8
1,I,5,12,3,7,2,10,5,5,4,13,3,9,2,8,4,10
2,D,4,11,6,8,6,10,6,2,6,10,3,7,3,7,3,9
3,N,7,11,6,6,3,5,9,4,6,4,4,10,6,10,2,8
4,G,2,1,3,1,1,8,6,6,6,6,5,9,1,7,5,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,D,2,2,3,3,2,7,7,7,6,6,6,4,2,8,3,7
19996,C,7,10,8,8,4,4,8,6,9,12,9,13,2,9,3,7
19997,T,6,9,6,7,5,6,11,3,7,11,9,5,2,12,2,4
19998,S,2,3,4,2,1,8,7,2,6,10,6,8,1,9,5,8


In [ ]:
X = df.drop('letter', axis=1)
y = df['letter']


In [ ]:
# Encoding

In [ ]:
le = LabelEncoder()

In [ ]:
y_encoded = le.fit_transform(y)

In [ ]:
# Scaling

In [ ]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# **Splitting data**

In [20]:
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded
)


In [ ]:
model = MLPClassifier(hidden_layer_sizes=(64, 32), activation='relu', max_iter=300, random_state=42)
model.fit(X_train, y_train)

/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPClassifier(hidden_layer_sizes=(64, 32), max_iter=300, random_state=42)

# Evaluation for base model

In [ ]:
y_pred = model.predict(X_test)
print("🔹 Base Model Classification Report:")
print(classification_report(y_test, y_pred, target_names=le.classes_))

🔹 Base Model Classification Report:
              precision    recall  f1-score   support

           A       0.97      0.98      0.98       158
           B       0.95      0.91      0.93       153
           C       0.97      0.97      0.97       147
           D       0.94      0.95      0.95       161
           E       0.95      0.94      0.94       154
           F       0.94      0.95      0.94       155
           G       0.96      0.94      0.95       155
           H       0.91      0.89      0.90       147
           I       0.91      0.92      0.91       151
           J       0.94      0.91      0.93       149
           K       0.92      0.94      0.93       148
           L       0.95      0.97      0.96       152
           M       0.97      0.97      0.97       158
           N       0.97      0.94      0.95       157
           O       0.97      0.96      0.96       150
           P       0.96      0.98      0.97       161
           Q       0.97      0.98      0.98  

# **Hyperparametr Tunnig**

In [ ]:
param_grid = {
    'hidden_layer_sizes': [(64,), (64, 32), (128, 64)],
    'activation': ['relu', 'tanh'],
    'solver': ['adam', 'sgd'],
    'learning_rate': ['constant', 'adaptive'],
    'alpha': [0.0001, 0.001],
    'max_iter': [300]
}

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform


In [ ]:

param_dist = {
    'hidden_layer_sizes': [(64,), (128,), (64, 32)],
    'activation': ['relu', 'tanh'],
    'solver': ['adam', 'sgd'],
    'learning_rate': ['constant', 'adaptive'],
    'alpha': uniform(0.0001, 0.01),  # random float values
    'max_iter': [100]
}


In [ ]:
random_search = RandomizedSearchCV(
    MLPClassifier(random_state=42),
    param_distributions=param_dist,
    n_iter=10,  # only test 10 random combinations
    cv=2,
    scoring='accuracy',
    n_jobs=-1,
    random_state=42
)

random_search.fit(X_train, y_train)

/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(


RandomizedSearchCV(cv=2, estimator=MLPClassifier(random_state=42), n_jobs=-1,
                   param_distributions={'activation': ['relu', 'tanh'],
                                        'alpha': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x78d498822cd0>,
                                        'hidden_layer_sizes': [(64,), (128,),
                                                               (64, 32)],
                                        'learning_rate': ['constant',
                                                          'adaptive'],
                                        'max_iter': [100],
                                        'solver': ['adam', 'sgd']},
                   random_state=42, scoring='accuracy')

In [22]:
y_best_pred = random_search.predict(X_test) # Predecting

# Evaluation for best model

In [ ]:
# Generate classification report
report = classification_report(y_test, y_best_pred, target_names=le.classes_)

report

'              precision    recall  f1-score   support\n\n           A       0.99      0.99      0.99       158\n           B       0.90      0.95      0.92       153\n           C       0.97      0.97      0.97       147\n           D       0.94      0.95      0.94       161\n           E       0.94      0.98      0.96       154\n           F       0.93      0.95      0.94       155\n           G       0.95      0.95      0.95       155\n           H       0.96      0.88      0.91       147\n           I       0.97      0.93      0.95       151\n           J       0.95      0.97      0.96       149\n           K       0.93      0.94      0.93       148\n           L       0.97      0.97      0.97       152\n           M       0.99      0.96      0.98       158\n           N       0.96      0.94      0.95       157\n           O       0.95      0.95      0.95       150\n           P       0.97      0.97      0.97       161\n           Q       0.98      0.97      0.97       157\n       